# Simple health economic DES model with Python

A very simple model to serve as an exploration of creating health economic Discrete-Event Simulation (DES) models in Python using SimPy library. The clinical context of the model is as follows:
* During a cycle of treatment, patients can either die (p = 0.2) or experience a full cycle of treatment without any other events occurring.
* Patients can receive up to five cycles of treatment.
* A full cycle causes a longer delay or timeout compared to dying during a cycle. These timeouts are assumed to generated by a exponential data generating process.
* After surviving the maximum number of treatment cycles, patients will enter a follow up phase, in which patients can also die.

The health economic context is as follows:
* Each cycle of treatment incurs 5000 euro initially and 250 euro per day.
* Followup costs 3500 euro per patient as a lumpsum.
* Health-related quality of life (QoL) in the treatment phase is 0.7 as expressed over the duration of one year.
* QoL in the followup phase is 0.8 as expressed over the duration of one year.

Furthermore, all patients are simulated at t = 0, so the interarrival time = 0. Note that the model currently only represents one comparator. So for an actual cost-effectiveness analysis, another comparator must be added, but this is very straightforward.

Two classes are created for the model: 1) a Patient class and 2) a Patient Generator class. The Patient class is the main component of the model. Here, patient characteristics are created as class attributes, and the model's logic or structure is programmed. The Patient Generator class is a generator of Patient instances, i.e., an object that simulates patients for our simulation.

In [1]:
# Remove the comment in the line below to install simpy if you're viewing this notebook in Google Colab. This cell needs to be run only once.
#%pip install simpy

In [4]:
# Importing libraries and setting the random seed for reproducibility.
from random import seed
import random
import simpy
import pandas as pd
import numpy as np
seed(123)

# Constants. By convention, these are written in capital letters.
MAX_CYCLES = 5 # Maximum number of treatment cycles per patient
PROB_DEATH = 0.1 #Probability of dying during one cycle per patient
N_PATIENTS = 1000 # Number of patients to simulate
C_TREATMENT_INIT = 5000
C_TREATMENT_DAILY = 250
C_FOLLOWUP = 3500
U_TREATMENT = 0.7
U_FOLLOWUP = 0.8
DAYS_PER_YEAR = 365.2422

class Patient:
    def __init__(self, env, patient_id, treatment_cycles = 0, cost = 0, utility = 0):
        
        """Initializing attributes of Patients. 
        Of course these can be expanded to more accurately reflect patient heterogeneity."""
        self.env = env
        self.patient_id = patient_id
        self.state = 'Alive'   # To use as a conditional
        self.treatment_cycles = treatment_cycles
        self.cost = cost
        self.utility = utility
        self.set_model_structure = env.process(self.set_model_structure(env)) # Used to start the process

    #################### SECTION: MODEL STRUCTURE ####################    

    def get_treatment(self, env):
        """ Method that models the treatment phase. 
        """
        while self.treatment_cycles < MAX_CYCLES and self.state == 'Alive':
            self.treatment_cycles += 1
            
            # First, the event that occurs during a cycle is determined.
            rand = random.uniform(0, 1)
            if (rand < PROB_DEATH):
                # EVENT: DEATH
                # CHANGE PATIENT'S STATE TO DEAD
                self.state = 'Dead'
                # SAMPLE A TIME-TO-EVENT
                time_to_death = np.random.gamma(1.5, 3, 1)
                # TIMEOUT EQUAL TO THE TIME-TO-EVENT
                yield env.timeout(time_to_death)
                # INCREMENT ACCUMULATED COSTS AND UTILITY
                self.cost += self.increment_cost(time_to_death)
                self.utility += self.increment_utility(time_to_death, U_TREATMENT)
#                print(f"Patient {self.patient_id} has died after {int(env.now)} days.")

            else:
                # EVENT: FULL CYCLE
                time_to_full_cycle = np.random.gamma(3, 10, 1)
                yield env.timeout(time_to_full_cycle)
                self.cost += self.increment_cost(time_to_full_cycle)
                self.utility += self.increment_utility(time_to_full_cycle, U_TREATMENT)
#                print(f"Patient {self.patient_id} has completed cycle {self.treatment_cycles} of treatment after {int(env.now)} days.")
            self.save_data(phase = 'treatment')
            
    def enter_followup(self, env):
        """ Method that models the follow up phase. 
        """ 
        if self.state == 'Alive':
            time_in_folllowup = np.random.gamma(2, 15, 1)
            yield env.timeout(time_in_folllowup)
#            print(f"Patient {self.patient_id} has completed follow up.")
            self.cost += C_FOLLOWUP
            self.utility += self.increment_utility(time_in_folllowup, U_FOLLOWUP)
            self.save_data(phase = 'followup')
            
    def set_model_structure(self, env):
        """This method is used to order individual care phases, such as a treatment and follow up phase.
        This method is the one that is actually executed, and calls the other methods."""
        yield env.process(self.get_treatment(env)) # Yielding or waiting on the the treatment phase to complete
        env.process(self.enter_followup(env)) # Before the follow up phase can commence.

    #################### END SECTION ####################
        
    #################### SECTION: HELPER METHODS ####################    
    def save_data(self, phase):
        """Append a list of outcomes of interest as specified here to a list that
        is created outside the Patient class. This method should be called whenever it
        is appropriate to save data. E.g., each treatment cycle.
        Note, appending a list to a list and converting the final list once to a dataframe is much more efficient."""
        output_list.append([self.patient_id, self.state, self.treatment_cycles, phase, self.cost, self.utility, env.now])

    def increment_cost(self, duration):
        """Helper method to increment the cost attribute of Patient"""
        cost_increment = int(duration) * C_TREATMENT_DAILY
        if self.treatment_cycles == 1:
            cost_increment += C_TREATMENT_INIT
        return cost_increment
    
    def increment_utility(self, duration, utility):
        """Helper method to increment the utility attribute of Patient"""
        utility_increment = duration * (utility/DAYS_PER_YEAR)
        return utility_increment
    #################### END SECTION ####################
        
            
class Patient_Generator(object):
    """A generator for patients that simulates 'max_arrivals' amount of patients
    or stops when time equals 'until'. This means that 'max_arrivals' amount
    of class instances of the Patient class are created. Upon creation,
    these Patient instances execute their model logic."""

    def __init__(self, env, until=simpy.core.Infinity,
                 max_arrivals = N_PATIENTS):
        self.env = env
        self.until = until      
        self.max_arrivals = max_arrivals

        self.num_patients_created = 0
        self.action = env.process(self.run())  # starts the run() method as a SimPy process

    def run(self):
        """The method that generates patients.
        """
        yield self.env.timeout(0)   # SimPy processes cannot be 'empty'
        # Main generator loop that terminates when enough patients are simulated or
        # when until is reached
        while self.env.now < self.until and self.num_patients_created < self.max_arrivals:
            self.num_patients_created += 1
            # Create a new instance of the Patient class
            Patient(env, patient_id = self.num_patients_created)

# Empty list to store outcomes of interest in. Used by the Patient.save_data() method.            
output_list = []

# Creating the simulation environment
env = simpy.Environment()

# Creating a single instance of class Patient_Generator
pat_gen = Patient_Generator(env)

# If desired, you can also manually create one patient as shown below
#pat = Patient(env, patient_id = 1)

# Running the simulation
env.run()

# Converting output_list to a pandas dataframe and set correct data types
output_df = pd.DataFrame(output_list, columns = ['patient_id', 'state', 'treatment_cycles', 'phase', 'cost', 'utility', 'simulation_time'])
output_df[['utility', 'simulation_time']] = output_df[['utility', 'simulation_time']].astype(float)

In [2]:
# Inspecting output
output_df.tail(n = 25)
output_df.loc[output_df['patient_id'] == 123]

df_summary = output_df.loc[output_df.groupby('patient_id')['simulation_time'].idxmax()]
df_summary = df_summary[['patient_id', 'cost', 'utility', 'simulation_time']]

df_summary[['cost', 'utility']].describe()

,cost,utility
count,1000.000000,1000.000000
mean,34312.750000,0.251597
std,17267.974197,0.153625
min,5000.000000,0.000377
25%,19687.500000,0.113946
50%,36375.000000,0.275277
75%,47250.000000,0.371116
max,83500.000000,0.709217


# Checking distributions

In [2]:
# Just some code to check the values of distributions for the model

shape = 1.5
scale = 3
x = np.random.gamma(shape, scale, 10000)

import matplotlib.pyplot as plt
import scipy.special as sps  
count, bins, ignored = plt.hist(x, 50, density=True)
y = bins**(shape-1)*(np.exp(-bins/scale) /  
                     (sps.gamma(shape)*scale**shape))
plt.plot(bins, y, linewidth=2, color='r')  
# plt.show()

NameError: name 'np' is not defined